<a href="https://colab.research.google.com/github/afykingsss/ML/blob/main/Prediksi_Kanker_Payudara_DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [29]:
#Memasukan dataset yang akan dimodelkan
df=pd.read_csv('breast-cancer-dataset.csv')
df


,S/N,Year,Age,Menopause,Tumor Size (cm),Inv-Nodes,Breast,Metastasis,Breast Quadrant,History,Diagnosis Result
0,1,2019,40,1,2,0,Right,0,Upper inner,0,Benign
1,2,2019,39,1,2,0,Left,0,Upper outer,0,Benign
2,3,2019,45,0,4,0,Left,0,Lower outer,0,Benign
3,4,2019,26,1,3,0,Left,0,Lower inner,1,Benign
4,5,2019,21,1,1,0,Right,0,Upper outer,1,Benign
...,...,...,...,...,...,...,...,...,...,...,...
208,209,2020,49,1,6,3,Right,1,Lower inner,1,Malignant
209,210,2020,28,1,3,0,Left,0,Upper inner,0,Benign
210,211,2020,22,1,1,0,Left,0,Upper outer,1,Benign
211,212,2020,19,1,1,0,Left,0,Lower inner,1,Benign


In [30]:
#Mengubah atribut / kolom 'Breast' &'Breast Quadrant" yang awalnya kategorik menjadi numerik.menggunakan get_dummies,yang akan menghasilkan atribut/kolom baru sesuai jumlah kategorik
states=pd.get_dummies(df['Breast'],drop_first=True)
states
states1=pd.get_dummies(df['Breast Quadrant'],drop_first=True)
states1
states=pd.concat([states,states1],axis=1)
#Variabel states yang berisi atribut/kolom hasil transformasi 'Breast' &'Breast Quadrant' digabungkan pada dataset utama
df=pd.concat([df,states],axis=1)
#menghapus atribut/kolom 'Breast' &'Breast Quadrant' yang masih kategorik
df=df.drop(['Breast','Breast Quadrant'],axis=1)



In [31]:
#Dikarenakan atribut/kolom 'Diagnosis Result' tidak berada ditengah dataframe,maka dipindahkan ke sudut kanan
diagnosis_result = df.pop('Diagnosis Result')
df.insert(len(df.columns), 'Diagnosis Result', diagnosis_result)


In [32]:
#Dikarenakan terdapat beberapa atribut/kolom yang memiliki data kosong yang diisi dengan string'#',maka string tersebut diubah
#menjadi NaN agar dapat dideteksi sebagai nilai kosong'null'.
df['Year'] = df['Year'].replace('#', np.nan)
df['Menopause'] = df['Menopause'].replace('#', np.nan)
df['Tumor Size (cm)'] = df['Tumor Size (cm)'].replace('#', np.nan)
df['Inv-Nodes'] = df['Inv-Nodes'].replace('#', np.nan)
df['Metastasis'] = df['Metastasis'].replace('#', np.nan)
df['History'] = df['History'].replace('#', np.nan)


In [33]:
#Baris yang berisi nilai kosong 'null' dihapus
df=df.dropna()
df

,S/N,Year,Age,Menopause,Tumor Size (cm),Inv-Nodes,Metastasis,History,Left,Right,Lower inner,Lower outer,Upper inner,Upper outer,Upper outer,Diagnosis Result
0,1,2019,40,1,2,0,0,0,False,True,False,False,True,False,False,Benign
1,2,2019,39,1,2,0,0,0,True,False,False,False,False,True,False,Benign
2,3,2019,45,0,4,0,0,0,True,False,False,True,False,False,False,Benign
3,4,2019,26,1,3,0,0,1,True,False,True,False,False,False,False,Benign
4,5,2019,21,1,1,0,0,1,False,True,False,False,False,True,False,Benign
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208,209,2020,49,1,6,3,1,1,False,True,True,False,False,False,False,Malignant
209,210,2020,28,1,3,0,0,0,True,False,False,False,True,False,False,Benign
210,211,2020,22,1,1,0,0,1,True,False,False,False,False,True,False,Benign
211,212,2020,19,1,1,0,0,1,True,False,True,False,False,False,False,Benign


In [34]:
#Mengidentifikasi atribut/kolom yang akan diuji/dimodelkan dengan labelnya
X=df.iloc[:,:-1]
y=df.iloc[:,15]


In [35]:
#Mencari 5 atribut/kolom yang paling berpengaruh terhadap label,menggunakan selection feature(Univariate Selection)
bestfeatures=SelectKBest(score_func=chi2,k=10)
fit=bestfeatures.fit(X,y)

dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
#concat two dataframes for better visualization
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Specs','Score']  #naming the dataframe columns
best_features = featureScores.nlargest(5, 'Score')
best_features

,Specs,Score
2,Age,304.007603
4,Tumor Size (cm),153.690940
5,Inv-Nodes,92.478134
6,Metastasis,76.967394
13,Upper outer,11.536600


In [36]:
#Menghapus kolom/atribut yang tidak digunakan
df=df.drop(['S/N','Year','Menopause','History','Left','Right','Lower inner','Lower outer','Upper inner'],axis=1)
df

,Age,Tumor Size (cm),Inv-Nodes,Metastasis,Upper outer,Upper outer,Diagnosis Result
0,40,2,0,0,False,False,Benign
1,39,2,0,0,True,False,Benign
2,45,4,0,0,False,False,Benign
3,26,3,0,0,False,False,Benign
4,21,1,0,0,True,False,Benign
...,...,...,...,...,...,...,...
208,49,6,3,1,False,False,Malignant
209,28,3,0,0,False,False,Benign
210,22,1,0,0,True,False,Benign
211,19,1,0,0,False,False,Benign


In [37]:
X=df.iloc[:,:-1]
y=df.iloc[:, 6]

In [38]:
#Membagi dataset kedalam perbandingan 70:30,70 sebagai data latih,30 sebagai data uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [39]:
#Melatih model dengan 70% dari data set,yang terdiri dari atribut dependent dan label
model = DecisionTreeClassifier(criterion = "entropy", random_state = 1234,max_depth = 5, min_samples_leaf = 10)
model.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=5, min_samples_leaf=10,
                       random_state=1234)

In [40]:
#Menguji model dengan 30% dataset yang belum diujikan
y_pred = model.predict(X_test)
y_pred

array(['Benign', 'Benign', 'Benign', 'Benign', 'Benign', 'Malignant',
       'Benign', 'Benign', 'Malignant', 'Malignant', 'Malignant',
       'Benign', 'Benign', 'Malignant', 'Benign', 'Malignant', 'Benign',
       'Malignant', 'Benign', 'Malignant', 'Benign', 'Malignant',
       'Benign', 'Malignant', 'Benign', 'Benign', 'Malignant',
       'Malignant', 'Benign', 'Benign', 'Benign', 'Malignant', 'Benign',
       'Malignant', 'Benign', 'Benign', 'Benign', 'Benign', 'Malignant',
       'Malignant', 'Benign', 'Benign', 'Malignant', 'Benign', 'Benign',
       'Benign', 'Malignant', 'Benign', 'Malignant', 'Malignant',
       'Malignant', 'Benign', 'Benign', 'Benign', 'Benign', 'Benign',
       'Benign', 'Benign', 'Malignant', 'Malignant', 'Malignant',
       'Malignant', 'Malignant', 'Malignant'], dtype=object)

In [41]:
#Akurasi dari model dalam mengkalsifikasi
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.90625


In [42]:
#Menguji Model dengan nilai yang belum pernah diujikan
uji=[[22,15,0,1,1,0],[40,2,0,0,0,0]]
prediksi=model.predict(uji)
prediksi

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array(['Malignant', 'Benign'], dtype=object)